https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.dynamic_factor_mq.DynamicFactorMQ.html
<br>https://medium.com/@philippetousignant/dynamic-factor-models-in-python-58d2d5252640
<br>https://www.ecb.europa.eu/pub/pdf/scpwps/ecbwp1564.pdf

In [ ]:
import datetime
import json
import pandas as pd
import numpy as np
import statsmodels.tsa.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

Static data

In [ ]:
files = {'sources':'Data Sources.csv', 'data':'Data Cleaned.json'}

In [ ]:
train_ratio = 0.8
max_factors, max_lags = 5, 2
max_ar = 9
forecast_steps = 1

Classes and functions

In [ ]:
class Variables:
    def __init__(self, df):
        self.dep = df[df['Dependent']=='Y']['Name'].tolist()[0]

        self.indep = df[df['Dependent']!='Y']['Name'].tolist()
        self.indep_exclude = ['business expectation']
        self.indep = [i for i in self.indep if all(j not in i.lower() for j in self.indep_exclude)]

        self.freq = dict(zip(df['Name'],df['Frequency']))

        self.non_stat = []

In [ ]:
def run_df(data, no_factors, f_lags):
    try:
        model = sm.DynamicFactorMQ(endog=data,
                                   k_endog_monthly=len([k for k,v in variables.freq.items() if v=='M']),
                                   factors=no_factors,
                                   factor_orders=f_lags,
                                   freq='M',
                                   idiosyncratic_ar1=True)
        results = model.fit(maxiter=100, disp=False)
        return(results)

    except UserWarning:
        print('-model did not converge-', end=' ')
        return(None)

In [ ]:
def find_min_ic(models_dict):
    # info criterion type: (model specifications, info criterion value)
    ic_min = {ic:(None,float('inf')) for ic in ics}

    # k: model specifications, v: model object
    for k,v in models_dict.items():
        for ic,val in ic_min.items():
            if v[ic]<ic_min[ic][1]:
                ic_min[ic] = (k,v[ic])
    return(ic_min)

In [ ]:
def compare(actual_df, predict_df):
    compare_df = pd.DataFrame(actual_df[variables.dep].dropna().values,
                              index=actual_df[variables.dep].dropna().index,
                              columns=['Actual'])

    compare_df = compare_df.merge(predict_df[[variables.dep]].rename(columns={variables.dep:'Predicted'}),
                                  how='left', left_index=True, right_index=True)
    return(compare_df)

In [ ]:
def split_train_test(data, train_prop):
    train_no_rows = round(train_prop * len(data))
    train_df = data.iloc[:train_no_rows]
    test_df = data.iloc[train_no_rows:]
    return(train_df, test_df)

In [ ]:
def run_train_test(full_data, train_data, forecast_steps, factor, lag):
    fc_df = pd.DataFrame()
    model = run_df(train_data, factor, lag)
    if not model:
        return(None) # exit func if model does not converge

    for i in range(-(len(full_data)-len(train_data)), 0, forecast_steps):
        model = model.apply(endog=full_data.iloc[:i])
        forecast = model.forecast(steps=forecast_steps)
        fc_df = pd.concat([fc_df, forecast])

    export = {'model':model, 'forecasts':fc_df}

    for ic in ics:
        export[ic] = getattr(model, ic)

    return(export)

Load raw data

In [ ]:
sources = pd.read_csv(files['sources'], encoding='utf-8')
variables = Variables(sources)

In [ ]:
with open(files['data']) as f:
    ts_data = json.load(f)

Convert json data to pandas series to check for unit roots and to resample

In [ ]:
ts_pd = {}
for series in ts_data:

    if variables.freq[series]=='Q':
        periods = [p.split()[0]+p.split()[-1][::-1] for p in ts_data[series]]
        periods = pd.PeriodIndex(periods, freq=variables.freq[series])
        ts_pd[series] = pd.Series(ts_data[series].values(), index=periods)

    elif variables.freq[series]=='M':
        # warning: pd version 2.0.0 has deprecated kwarg infer_datetime_format
        periods = pd.to_datetime(list(ts_data[series]),infer_datetime_format=True) + pd.tseries.offsets.MonthEnd(0)
        periods = periods.date
        ts_pd[series] = pd.Series(ts_data[series].values(), index=periods)

    elif variables.freq[series]=='D':
        periods = pd.to_datetime(list(ts_data[series])).date
        periods = pd.PeriodIndex(periods, freq=variables.freq[series])
        ts_pd[series] = pd.Series(ts_data[series].values(), index=periods).resample('M').last()
        variables.freq[series] = 'M' # update to 'M' since resampled

    # start all series from first valid index
    ts_pd[series] = ts_pd[series].sort_index()
    ts_pd[series] = ts_pd[series][ts_pd[series].first_valid_index():]

In [ ]:
# if p-value >0.05, variable is non-stationary
for i in variables.indep:
    print(i)
    # if unit root, take % yoy growth (which also removes seasonality)
    if adfuller(ts_pd[i])[1]>0.05:

        if variables.freq[i]=='M' and 'sora' in i.lower():
            ts_pd[i] = ts_pd[i].diff(periods=12)
        elif variables.freq[i]=='M':
            ts_pd[i] = ts_pd[i].pct_change(periods=12) * 100
        elif variables.freq[i]=='Q':
            ts_pd[i] = ts_pd[i].pct_change(periods=4) * 100

        ts_pd[i] = ts_pd[i][ts_pd[i].first_valid_index():]
        print('Non-stationary', end='\n\n')
        variables.non_stat.append(i)

    else:
        print('Stationary', end='\n\n')

In [ ]:
# resample all series to monthly and start from first valid index
for series,freq in variables.freq.items():

    if freq=='Q':
        ts_pd[series] = ts_pd[series].resample('M', convention='end').asfreq()

    try:
        ts_pd[series].index = ts_pd[series].index.to_timestamp() + pd.offsets.MonthEnd(0)
    except:
        pass

    ts_pd[series].index = ts_pd[series].index + pd.offsets.MonthEnd(0)

Convert pd series dictionary to dataframe

Column order adhered to as defined by statsmodels docs for dynamic factor modelling
<br>https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.dynamic_factor_mq.DynamicFactorMQ.html
- dependent variable leftmost
- monthly data in the first columns
- quarterly data in the last columns

In [ ]:
# pass all series into dataframe, start dataframe from first year of GDP growth data
ts_df = pd.DataFrame(ts_pd)
ts_df = ts_df.loc[ts_df.index.year>=ts_df.loc[:,variables.dep].first_valid_index().year]

# rearrange columns in correct order for factor modelling as explained in markdown above
ts_df = ts_df[[variables.dep]+\
              [k for k,v in variables.freq.items() if v=='M' and k!=variables.dep]+\
              [k for k,v in variables.freq.items() if v=='Q' and k!=variables.dep]]

ts_df.index.name = 'Period'
ts_df

Explore data

In [ ]:
ts_df.describe().round(3)

In [ ]:
#with plt.rc_context(rc={'figure.max_open_warning':0}):
#    for i in list(ts_df):
#        plt.figure(figsize=(6,4))
#        plt.title('\n'.join('\n'.join(i.split(' | ')).split(' - ')), fontdict={'fontsize':10})
#
#        # fillna because quarterly data has blanks when freq=monthly
#        plt_data = ts_df[[i]].loc[ts_df[[i]].first_valid_index():ts_df[[i]].last_valid_index()].fillna(method='bfill',limit=2)
#
#        plt.plot(plt_data)
#        plt.tight_layout()

Instantiate and fit DF model

In [ ]:
# to catch warnings for models that do not converge
warnings.filterwarnings('error', category=UserWarning)

In [ ]:
ics = ['aic','bic','hqic']

In [ ]:
models = {}
for factor in range(1, max_factors+1):
    for lag in range(1, max_lags+1):
        print(f'({factor}, {lag})', end=' ')
        models[(factor,lag)] = {'model':run_df(ts_df, factor, lag)}

        # if model converges, extract info criterions. if not, remove model from dict.
        if models[(factor,lag)]['model']:
            for ic in ics:
                models[(factor,lag)][ic] = getattr(models[(factor, lag)]['model'], ic)
        else:
            models.pop((factor,lag))
        print('done')

In [ ]:
models.keys()

For each information criterion-model pair, run predictions. Then choose model that has lowest RMSE.

In [ ]:
models_min_ic = find_min_ic(models)

In [ ]:
best_model = (None, np.inf)
for i in ics:
    use_model = models[models_min_ic[i][0]]
    compare_df = compare(ts_df, use_model['model'].predict())
    mean_sq_err = mse(compare_df['Actual'], compare_df['Predicted'])

    if mean_sq_err<best_model[1]:
        best_model = (models_min_ic[i][0], mean_sq_err)

    print(f"[{i}] In-sample MSE of GDP forecast: {round(mean_sq_err,3)}")

    plt.figure(figsize=(6,4))
    plt.title(label=f"{variables.dep}\nDynamic Factor Model\n\
                    (Factors: {models_min_ic[i][0][0]}, Order: {models_min_ic[i][0][1]})")

    plt.plot(compare_df.iloc[-(4*10):])
    plt.tight_layout()
    plt.show()

In [ ]:
best_model

Pseudo out-of-sample forecasts, i.e., train on x% of dataset, test on (1-x)%

In [ ]:
train_ts, test_ts = split_train_test(ts_df, train_ratio)

In [ ]:
oos_models = {}
with np.errstate(divide='ignore'):
    for factor in range(1, max_factors+1):
        for lag in range(1, max_lags+1):
            print(f'({factor}, {lag})', end=' ')
            oos_models[(factor,lag)] = run_train_test(ts_df, train_ts, forecast_steps, factor, lag)

            # if model does not converge, remove from dict
            if not oos_models[(factor,lag)]:
                oos_models.pop((factor,lag))
            print('done')

In [ ]:
oos_models.keys()

In [ ]:
oos_models_min_ic = find_min_ic(oos_models)
oos_models_min_ic

In [ ]:
best_oos_model = (None, np.inf)
for i in ics:
    use_oos_model = oos_models[oos_models_min_ic[i][0]]
    compare_df = compare(ts_df, use_oos_model['model'].predict())
    mean_sq_err = mse(compare_df['Actual'], compare_df['Predicted'])

    if mean_sq_err<best_oos_model[1]:
        best_oos_model = (oos_models_min_ic[i][0], mean_sq_err)

    print(f"[{i}] Out-sample MSE of GDP forecast: {round(mean_sq_err,3)}")

    plt.figure(figsize=(6,4))
    plt.title(label=f"{variables.dep}\nDynamic Factor Model\n\
                    (Factors: {oos_models_min_ic[i][0][0]}, Order: {oos_models_min_ic[i][0][1]})")

    plt.plot(compare_df.iloc[-(4*10):])
    plt.tight_layout()
    plt.show()

In [ ]:
best_oos_model

Run an AR model as a baseline

In [ ]:
ar_df = ts_df[[variables.dep]].dropna()

In [ ]:
if adfuller(ar_df[variables.dep])[1]>0.05:
    print(f'[Non-stationary] {variables.dep}')
else:
    print(f'[Stationary] {variables.dep}')

In [ ]:
ar_models = {(p,0,0):None for p in range(1,max_ar)}

In [ ]:
for i in ar_models:
    print(f'{i}', end=' ')
    ar_models[i] = {'model':ARIMA(ar_df, order=i).fit()}
    for ic in ics:
        ar_models[i][ic] = getattr(ar_models[i]['model'], ic)
    print('done')

In [ ]:
ar_models_min_ic = find_min_ic(ar_models)
ar_models_min_ic

In [ ]:
best_ar_model = (None, np.inf)
for i in ar_models_min_ic:
    use_ar_model = ar_models[ar_models_min_ic[i][0]]

    pred_ar = pd.DataFrame(use_ar_model['model'].predict())
    pred_ar.columns = [variables.dep]

    compare_df = compare(ts_df, pred_ar)

    mean_sq_err = mse(compare_df['Actual'], compare_df['Predicted'])

    if mean_sq_err<best_ar_model[1]:
        best_ar_model = (ar_models_min_ic[i][0], mean_sq_err)

    print(f"[{i}] In-sample MSE of GDP forecast: {round(mean_sq_err,3)}")

    plt.figure(figsize=(6,4))
    plt.title(label=f"{variables.dep}\nARIMA\n\
                    p/lags: {ar_models_min_ic[i][0][0]}, d/diff: {ar_models_min_ic[i][0][1]}, q/moving-avg: {ar_models_min_ic[i][0][2]}")

    plt.plot(compare_df.iloc[-(4*10):])
    plt.tight_layout()
    plt.show()

In [ ]:
best_ar_model

Export forecasts to csv

In [ ]:
export_df = ts_df[[variables.dep]].dropna().copy()
export_df.columns = ['Actual']

dynam_pred = models[best_model[0]]['model'].predict()[[variables.dep]]
dynam_pred.columns = [f"Dynamic Factor {best_model[0]}"]

export_ar = pred_ar.copy()
export_ar.columns = [f"ARIMA {best_ar_model[0]}"]

export_df = export_df.merge(dynam_pred, left_index=True, right_index=True)
export_df = export_df.merge(export_ar, left_index=True, right_index=True)
export_df

In [ ]:
export_df.to_csv('Classical Statistics Model Predictions.csv', encoding='utf-8')